#*Import Library*


> Import library berfungsi untuk memanggil kumpulan kode (fungsi, class, modul) yang sudah tersedia agar dapat digunakan di program, sehingga mempermudah, mempercepat, dan memperkaya fitur tanpa harus menulis kode dari awal.




In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# *Import Dataset*


> Import dataset adalah proses memasukkan data dari sumber eksternal (seperti file CSV, Excel, SQL, dll.) ke dalam program agar data tersebut bisa diolah atau dianalisis.



In [2]:
!git clone https://github.com/balrks/linear-poly-rice-model.git

fatal: destination path 'linear-poly-rice-model' already exists and is not an empty directory.


In [3]:
df = pd.read_csv('/content/linear-poly-rice-model/data_harga_beras_2020_2024.csv')
df

,Date,Semua Provinsi,Aceh,Kota Lhokseumawe
0,2020-01-01,11800.0,10300.0,10000.0
1,2020-01-02,11800.0,10300.0,10000.0
2,2020-01-03,11850.0,10300.0,10000.0
3,2020-01-06,11800.0,10350.0,10000.0
4,2020-01-07,11800.0,10350.0,10000.0
...,...,...,...,...
1213,2024-08-26,15350.0,13900.0,13450.0
1214,2024-08-27,15350.0,13900.0,13450.0
1215,2024-08-28,15250.0,13900.0,13450.0
1216,2024-08-29,15300.0,13900.0,13450.0


In [4]:
# Load the data from the CSV file
df = pd.read_csv('/content/linear-poly-rice-model/data_harga_beras_2020_2024.csv')

# Mengubah kolom 'date' menjadi format datetime
df['Date'] = pd.to_datetime(df['Date'])

# *Pre-processing Data*

> Preprocessing data adalah tahap membersihkan, mengubah, dan menyiapkan data mentah agar siap digunakan untuk analisis atau pemodelan.



In [5]:
# Mengurutkan DataFrame berdasarkan kolom 'Date'
df = df.sort_values(by='Date')
df['Date'] = range(1, len(df) + 1)

# Mengonversi kolom 'Date' menjadi tipe data integer (int64)
df['Date'] = df['Date'].astype('int64')

In [6]:
# Check missing values
df.isnull().sum()

,0
Date,0
Semua Provinsi,0
Aceh,0
Kota Lhokseumawe,0


# *Data Splitting*

> Data Splitting atau pemisahan data adalah proses membagi data menjadi dua bagian atau lebih. Tahap ini melibatkan pemisahan variabel independen dan dependen, Dalam pemisahan data, dataset dibagi menjadi set pelatihan (training) yang lebih besar dan set pengujian (testing) yang lebih kecil



In [7]:
# Prepare data for regression
X = df['Date'].values.reshape(-1, 1)  # Independent variable
Y = df['Kota Lhokseumawe'].values     # Dependent variable

In [8]:
# Splitting data into training data and testing data.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
print('Shape of X_train :', X_train.shape)
print('Shape of Y_train :', Y_train.shape)
print('Shape of X_test :', X_test.shape)
print('Shape of Y_test :', Y_test.shape)

Shape of X_train : (974, 1)
Shape of Y_train : (974,)
Shape of X_test : (244, 1)
Shape of Y_test : (244,)


# *Model Building*

> Proses pembuatan model dilakukan dengan menggunakan data training, Dalam hal ini pengolahan model pada metode Regresi Linear dan Regresi Polinomial



In [10]:
# Fit the Linear Regression model on the train data
linear = LinearRegression()
model_linear = linear.fit(X_train, Y_train)

# Print the coefficients and the intercept of the model
print('Nilai intercept (a): ',linear.intercept_)
print('Nilai slope (b): ',linear.coef_)

Nilai intercept (a):  8805.79649552436
Nilai slope (b):  [3.62721857]


In [11]:
# Initialize PolynomialFeatures
poly_2 = PolynomialFeatures(degree=2)

X_train_poly_2 = poly_2.fit_transform(X_train)

poly_2_model = LinearRegression()
poly_2_model.fit(X_train_poly_2, Y_train)

# Print the coefficients and the intercept
print('Nilai Intercept:', poly_2_model.intercept_)
print("Nilai slope:", poly_2_model.coef_)

Nilai Intercept: 9947.783130437068
Nilai slope: [ 0.         -2.0219701   0.00463403]


In [12]:
# Initialize PolynomialFeatures
poly_3 = PolynomialFeatures(degree=3)

X_train_poly_3 = poly_3.fit_transform(X_train)

poly_3_model = LinearRegression()
poly_3_model.fit(X_train_poly_3, Y_train)

# Print the coefficients and the intercept
print('Nilai Intercept:', poly_3_model.intercept_)
print("Nilai slope:", poly_3_model.coef_)


Nilai Intercept: 10293.254164805698
Nilai slope: [ 0.00000000e+00 -5.47311619e+00  1.17225001e-02 -3.87524452e-06]


In [13]:
# Initialize PolynomialFeatures
poly_4 = PolynomialFeatures(degree=4)

X_train_poly_4 = poly_4.fit_transform(X_train)

poly_4_model = LinearRegression()
poly_4_model.fit(X_train_poly_4, Y_train)

# Print the coefficients and the intercept
print('Nilai Intercept:', poly_4_model.intercept_)
print("Nilai slope:", poly_4_model.coef_)


Nilai Intercept: 9731.313125044031
Nilai slope: [ 0.00000000e+00  3.98026709e+00 -2.34633746e-02  4.12228435e-05
 -1.85505646e-08]


In [14]:
# Initialize PolynomialFeatures
poly_5 = PolynomialFeatures(degree=5)

X_train_poly_5 = poly_5.fit_transform(X_train)

poly_5_model = LinearRegression()
poly_5_model.fit(X_train_poly_5, Y_train)

# Print the coefficients and the intercept
print('Nilai Intercept:', poly_5_model.intercept_)
print("Nilai slope:", poly_5_model.coef_)


Nilai Intercept: 9902.910107716594
Nilai slope: [ 0.00000000e+00 -7.29617638e-06 -1.47187878e-03 -5.64689117e-06
  2.39139853e-08 -1.37466803e-11]


# Prediction

> Setelah model Regresi Linear dan Regresi Polinomial dibangun, tahap prediksi dilaksanakan dengan menerapkan kedua model tersebut pada data testing. Dalam proses ini, masing-masing model digunakan untuk memprediksi harga beras pada periode waktu yang sesuai dengan data testing.



In [15]:
# Gabungkan data X_test dan Y_test untuk analisis data uji
lin_pred = pd.DataFrame(X_test, columns=['X'])  # pakai X_test langsung
lin_pred['Harga Beras (Y)'] = Y_test
lin_pred['Prediksi'] = model_linear.predict(X_test)

# Tampilkan data testing
print(lin_pred)

        X  Harga Beras (Y)      Prediksi
0     542          10300.0  10771.748960
1     260          10000.0   9748.873324
2      44           9400.0   8965.394113
3    1009          13200.0  12465.660032
4     585          10150.0  10927.719359
..    ...              ...           ...
239   421           9800.0  10332.855513
240   244           9900.0   9690.837827
241    60           9600.0   9023.429610
242  1074          13450.0  12701.429239
243    64           9650.0   9037.938484

[244 rows x 3 columns]


In [16]:
# Prediksi menggunakan model regresi linear
Y_pred_linear = model_linear.predict(X_test)

# Membuat plot dengan Plotly
fig = go.Figure()

# Tambahkan scatter plot untuk data aktual
fig.add_trace(go.Scatter(
    x=X_test.flatten(),  # langsung pakai X_test
    y=Y_test, mode='markers',
    marker=dict(color='blue', size=6, opacity=0.6),
    name='Actual Prices'
))

# Tambahkan garis regresi linear
fig.add_trace(go.Scatter(
    x=X_test.flatten(),  # langsung pakai X_test
    y=Y_pred_linear, mode='lines',
    line=dict(color='red', width=2),
    name='Linear Regression Prediction'
))

# Konfigurasi tata letak plot
fig.update_layout(
    title=dict(text="Linear Regression Predictions", x=0.5),
    xaxis=dict(title="Independent Variabel", tickangle=-45, showgrid=True, gridcolor='lightgrey'),  # karena X bukan tanggal lagi
    yaxis=dict(title="Rice Price", tickformat='d', showgrid=True, gridcolor='lightgrey'),
    width=1000, height=600,
    plot_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12),
    shapes=[dict(
        type="rect", x0=0, y0=0, x1=1, y1=1, xref="paper", yref="paper",
        line=dict(color="black", width=1)
    )],
    legend=dict(
        x=0.5, y=1.1, xanchor="center", yanchor="top", orientation="h",
    )
)

fig.show()


In [17]:
# Gabungkan data X_test dan Y_test untuk analisis data uji
poly_2_pred = pd.DataFrame(X_test, columns=['X'])  # pakai X_test langsung
poly_2_pred['Harga Beras (Y)'] = Y_test
poly_2_pred['Prediksi'] = poly_2_model.predict(poly_2.transform(X_test))

# Tampilkan data testing
print(poly_2_pred)

        X  Harga Beras (Y)      Prediksi
0     542          10300.0  10213.186049
1     260          10000.0   9735.331223
2      44           9400.0   9867.787925
3    1009          13200.0  12625.431542
4     585          10150.0  10350.810984
..    ...              ...           ...
239   421           9800.0   9917.873542
240   244           9900.0   9730.313940
241    60           9600.0   9843.147427
242  1074          13450.0  13121.427757
243    64           9650.0   9837.358024

[244 rows x 3 columns]


In [18]:
# Gabungkan data X_test dan Y_test untuk analisis data uji
poly_3_pred = pd.DataFrame(X_test, columns=['X'])  # pakai X_test langsung
poly_3_pred['Harga Beras (Y)'] = Y_test
poly_3_pred['Prediksi'] = poly_3_model.predict(poly_3.transform(X_test))

# Tampilkan data testing
print(poly_3_pred)

        X  Harga Beras (Y)      Prediksi
0     542          10300.0  10153.456939
1     260          10000.0   9594.573666
2      44           9400.0  10074.801704
3    1009          13200.0  12724.513930
4     585          10150.0  10327.383544
..    ...              ...           ...
239   421           9800.0   9777.615109
240   244           9900.0   9599.429741
241    60           9600.0  10006.231141
242  1074          13450.0  13135.968249
243    64           9650.0   9989.974217

[244 rows x 3 columns]


In [19]:
# Gabungkan data X_test dan Y_test untuk analisis data uji
poly_4_pred = pd.DataFrame(X_test, columns=['X'])  # pakai X_test langsung
poly_4_pred['Harga Beras (Y)'] = Y_test
poly_4_pred['Prediksi'] = poly_4_model.predict(poly_4.transform(X_test))

# Tampilkan data testing
print(poly_4_pred)

        X  Harga Beras (Y)      Prediksi
0     542          10300.0   9958.564489
1     260          10000.0   9819.819520
2      44           9400.0   9864.461781
3    1009          13200.0  12978.239548
4     585          10150.0  10110.292168
..    ...              ...           ...
239   421           9800.0   9741.564405
240   244           9900.0   9838.665042
241    60           9600.0   9894.324721
242  1074          13450.0  13328.254785
243    64           9650.0   9900.439331

[244 rows x 3 columns]


In [20]:
# Gabungkan data X_test dan Y_test untuk analisis data uji
poly_5_pred = pd.DataFrame(X_test, columns=['X'])  # pakai X_test langsung
poly_5_pred['Harga Beras (Y)'] = Y_test
poly_5_pred['Prediksi'] = poly_5_model.predict(poly_5.transform(X_test))

# Tampilkan data testing
print(poly_5_pred)

        X  Harga Beras (Y)      Prediksi
0     542          10300.0   9992.159449
1     260          10000.0   9797.107671
2      44           9400.0   9899.666569
3    1009          13200.0  13013.741840
4     585          10150.0  10127.592212
..    ...              ...           ...
239   421           9800.0   9790.104230
240   244           9900.0   9806.122304
241    60           9600.0   9896.690414
242  1074          13450.0  13383.774394
243    64           9650.0   9895.786976

[244 rows x 3 columns]


In [21]:
# Membuat rentang nilai X yang lebih halus
X_smooth = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)

# Buat objek polynomial degree 2
poly_2 = PolynomialFeatures(degree=2)

# Transformasikan X_smooth
X_smooth_poly_2 = poly_2.fit_transform(X_smooth)

# Prediksi menggunakan model polinomial degree 2
Y_pred_poly_2_smooth = poly_2_model.predict(X_smooth_poly_2)

# Membuat figure dengan Plotly
fig = go.Figure()

# Scatter data asli
fig.add_trace(go.Scatter(
    x=X.flatten(),
    y=Y,
    mode='markers',
    marker=dict(color='blue', size=6, opacity=0.6),
    name='Data Asli'
))

# Garis prediksi polinomial degree 2 (smooth)
fig.add_trace(go.Scatter(
    x=X_smooth.flatten(),
    y=Y_pred_poly_2_smooth,
    mode='lines',
    line=dict(color='green', width=2),
    name='Prediksi Polinomial (Degree 2)'
))

# Layout plot
fig.update_layout(
    title=dict(text="Prediksi Harga Beras dengan Regresi Polinomial Degree 2", x=0.5),
    xaxis=dict(title="Independent Variabel", tickangle=-45, showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(title="Harga Beras", tickformat='d', showgrid=True, gridcolor='lightgrey'),
    width=1000,
    height=600,
    plot_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12),
    shapes=[dict(
        type="rect", x0=0, y0=0, x1=1, y1=1, xref="paper", yref="paper",
        line=dict(color="black", width=1)
    )],
    legend=dict(
        x=0.5, y=1.1, xanchor="center", yanchor="top", orientation="h"
    )
)

# Tampilkan plot
fig.show()


In [22]:
X_smooth = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
poly_3 = PolynomialFeatures(degree=3)
X_smooth_poly_3 = poly_3.fit_transform(X_smooth)
Y_pred_poly_3_smooth = poly_3_model.predict(X_smooth_poly_3)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=X.flatten(),
    y=Y,
    mode='markers',
    marker=dict(color='blue', size=6, opacity=0.6),
    name='Data Asli'
))

fig.add_trace(go.Scatter(
    x=X_smooth.flatten(),
    y=Y_pred_poly_3_smooth,
    mode='lines',
    line=dict(color='green', width=2),
    name='Prediksi Polinomial (Degree 3)'
))

fig.update_layout(
    title=dict(text="Prediksi Harga Beras dengan Regresi Polinomial Degree 3", x=0.5),
    xaxis=dict(title="Independent Variabel", tickangle=-45, showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(title="Harga Beras", tickformat='d', showgrid=True, gridcolor='lightgrey'),
    width=1000,
    height=600,
    plot_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12),
    shapes=[dict(
        type="rect", x0=0, y0=0, x1=1, y1=1, xref="paper", yref="paper",
        line=dict(color="black", width=1)
    )],
    legend=dict(
        x=0.5, y=1.1, xanchor="center", yanchor="top", orientation="h"
    )
)
fig.show()


In [23]:
X_smooth = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
poly_4 = PolynomialFeatures(degree=4)
X_smooth_poly_4 = poly_4.fit_transform(X_smooth)
Y_pred_poly_4_smooth = poly_4_model.predict(X_smooth_poly_4)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=X.flatten(),
    y=Y,
    mode='markers',
    marker=dict(color='blue', size=6, opacity=0.6),
    name='Data Asli'
))

fig.add_trace(go.Scatter(
    x=X_smooth.flatten(),
    y=Y_pred_poly_4_smooth,
    mode='lines',
    line=dict(color='green', width=2),
    name='Prediksi Polinomial (Degree 4)'
))

fig.update_layout(
    title=dict(text="Prediksi Harga Beras dengan Regresi Polinomial Degree 4", x=0.5),
    xaxis=dict(title="Independent Variabel", tickangle=-45, showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(title="Harga Beras", tickformat='d', showgrid=True, gridcolor='lightgrey'),
    width=1000,
    height=600,
    plot_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12),
    shapes=[dict(
        type="rect", x0=0, y0=0, x1=1, y1=1, xref="paper", yref="paper",
        line=dict(color="black", width=1)
    )],
    legend=dict(
        x=0.5, y=1.1, xanchor="center", yanchor="top", orientation="h"
    )
)
fig.show()


In [24]:
X_smooth = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
poly_5 = PolynomialFeatures(degree=5)
X_smooth_poly_5 = poly_5.fit_transform(X_smooth)
Y_pred_poly_5_smooth = poly_5_model.predict(X_smooth_poly_5)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=X.flatten(),
    y=Y,
    mode='markers',
    marker=dict(color='blue', size=6, opacity=0.6),
    name='Data Asli'
))

fig.add_trace(go.Scatter(
    x=X_smooth.flatten(),
    y=Y_pred_poly_4_smooth,
    mode='lines',
    line=dict(color='green', width=2),
    name='Prediksi Polinomial (Degree 5)'
))

fig.update_layout(
    title=dict(text="Prediksi Harga Beras dengan Regresi Polinomial Degree 5", x=0.5),
    xaxis=dict(title="Independent Variabel", tickangle=-45, showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(title="Harga Beras", tickformat='d', showgrid=True, gridcolor='lightgrey'),
    width=1000,
    height=600,
    plot_bgcolor='white',
    font=dict(family="Arial, sans-serif", size=12),
    shapes=[dict(
        type="rect", x0=0, y0=0, x1=1, y1=1, xref="paper", yref="paper",
        line=dict(color="black", width=1)
    )],
    legend=dict(
        x=0.5, y=1.1, xanchor="center", yanchor="top", orientation="h"
    )
)
fig.show()


#*Model Testing*

> Tahap pengujian membandingkan prediksi model dengan nilai aktual harga beras pada data testing, menggunakan tiga metrik evaluasi yaitu Mean Absolute Error (MAE) Root Mean Squared Error (RMSE) dan Mean Absolute Percentage Error (MAPE)



In [25]:
# Prediksi regresi linear
Y_pred_linear = model_linear.predict(X_test)

# Hitung metrik evaluasi untuk regresi linear
mae_linear = mean_absolute_error(Y_test, Y_pred_linear)
rmse_linear = np.sqrt(np.mean((Y_test - Y_pred_linear) ** 2))
mape_linear = np.mean(np.abs((Y_test - Y_pred_linear) / Y_test)) * 100

# Cetak hasil evaluasi regresi linear
print("Regresi Linear")
print(f"  MAE : {mae_linear:.4f}")
print(f"  RMSE: {rmse_linear:.4f}")
print(f"  MAPE: {mape_linear:.2f}%\n")


Regresi Linear
  MAE : 554.0741
  RMSE: 614.2957
  MAPE: 5.16%



In [26]:
# Mapping model dan poly transformernya
models = {
    2: (poly_2_model, poly_2),
    3: (poly_3_model, poly_3),
    4: (poly_4_model, poly_4),
    5: (poly_5_model, poly_5)
}

# Loop untuk prediksi dan evaluasi
for degree, (model, poly) in models.items():
    # The problem was here. Changed X_test to X_train
    Y_pred = model.predict(poly.transform(X_test))

    mae = mean_absolute_error(Y_test, Y_pred)
    rmse = np.sqrt(np.mean((Y_test - Y_pred) ** 2))
    mape = np.mean(np.abs((Y_test - Y_pred) / Y_test)) * 100

    print(f"Polynomial Degree {degree}")
    print(f"  MAE : {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%\n")

Polynomial Degree 2
  MAE : 286.50
  RMSE: 359.55
  MAPE: 2.54%

Polynomial Degree 3
  MAE : 280.43
  RMSE: 345.92
  MAPE: 2.53%

Polynomial Degree 4
  MAE : 205.23
  RMSE: 284.88
  MAPE: 1.85%

Polynomial Degree 5
  MAE : 210.25
  RMSE: 281.02
  MAPE: 1.90%



# *Future Prediction*
Memprediksi Masa depan berdasarkan model yang sudah dibuat

In [27]:
# 1. Cari X terakhir
X_last = X.max()

# 2. Buat 10 hari ke depan
X_future = np.linspace(X_last + 1, X_last + 10, 10).reshape(-1, 1)

# 3. Prediksi menggunakan model regresi linear
Y_future_pred_linear = model_linear.predict(X_future)

# 4. Tampilkan hasil prediksi
print("Prediksi Harga Beras 10 Hari ke Depan (Regresi Linear):")
for i, harga in enumerate(Y_future_pred_linear, start=1):
    print(f"Hari ke-{i}: {harga:.2f}")


Prediksi Harga Beras 10 Hari ke Depan (Regresi Linear):
Hari ke-1: 13227.38
Hari ke-2: 13231.00
Hari ke-3: 13234.63
Hari ke-4: 13238.26
Hari ke-5: 13241.88
Hari ke-6: 13245.51
Hari ke-7: 13249.14
Hari ke-8: 13252.77
Hari ke-9: 13256.39
Hari ke-10: 13260.02


In [28]:
# 1. Cari X terakhir
X_last = X.max()

# 2. Buat 10 hari ke depan
X_future = np.linspace(X_last + 1, X_last + 10, 10).reshape(-1, 1)

# 3. Transformasi X_future ke polynomial degree 2
X_future_poly = poly_2.transform(X_future)

# 4. Prediksi harga masa depan
Y_future_pred = poly_2_model.predict(X_future_poly)

# 5. Tampilkan hasil prediksi
print("Prediksi Harga Beras 10 Hari ke Depan:")
for i, harga in enumerate(Y_future_pred, start=1):
    print(f"Hari ke-{i}: {harga:.2f}")


Prediksi Harga Beras 10 Hari ke Depan:
Hari ke-1: 14368.99
Hari ke-2: 14378.27
Hari ke-3: 14387.56
Hari ke-4: 14396.86
Hari ke-5: 14406.16
Hari ke-6: 14415.48
Hari ke-7: 14424.81
Hari ke-8: 14434.14
Hari ke-9: 14443.49
Hari ke-10: 14452.84


In [29]:
X_last = X.max()

X_future = np.linspace(X_last + 1, X_last + 10, 10).reshape(-1, 1)

X_future_poly = poly_3.transform(X_future)

# 4. Prediksi harga masa depan
Y_future_pred = poly_3_model.predict(X_future_poly)

print("Prediksi Harga Beras 10 Hari ke Depan:")
for i, harga in enumerate(Y_future_pred, start=1):
    print(f"Hari ke-{i}: {harga:.2f}")


Prediksi Harga Beras 10 Hari ke Depan:
Hari ke-1: 14021.14
Hari ke-2: 14026.97
Hari ke-3: 14032.79
Hari ke-4: 14038.61
Hari ke-5: 14044.42
Hari ke-6: 14050.23
Hari ke-7: 14056.03
Hari ke-8: 14061.83
Hari ke-9: 14067.63
Hari ke-10: 14073.42


In [30]:
X_last = X.max()

X_future = np.linspace(X_last + 1, X_last + 10, 10).reshape(-1, 1)

X_future_poly = poly_4.transform(X_future)

# 4. Prediksi harga masa depan
Y_future_pred = poly_4_model.predict(X_future_poly)

print("Prediksi Harga Beras 10 Hari ke Depan:")
for i, harga in enumerate(Y_future_pred, start=1):
    print(f"Hari ke-{i}: {harga:.2f}")


Prediksi Harga Beras 10 Hari ke Depan:
Hari ke-1: 13426.97
Hari ke-2: 13423.06
Hari ke-3: 13419.08
Hari ke-4: 13415.03
Hari ke-5: 13410.89
Hari ke-6: 13406.68
Hari ke-7: 13402.39
Hari ke-8: 13398.02
Hari ke-9: 13393.58
Hari ke-10: 13389.05


In [31]:
X_last = X.max()

X_future = np.linspace(X_last + 1, X_last + 10, 10).reshape(-1, 1)

X_future_poly = poly_5.transform(X_future)

# 4. Prediksi harga masa depan
Y_future_pred = poly_5_model.predict(X_future_poly)

print("Prediksi Harga Beras 10 Hari ke Depan:")
for i, harga in enumerate(Y_future_pred, start=1):
    print(f"Hari ke-{i}: {harga:.2f}")


Prediksi Harga Beras 10 Hari ke Depan:
Hari ke-1: 13289.79
Hari ke-2: 13282.47
Hari ke-3: 13275.04
Hari ke-4: 13267.49
Hari ke-5: 13259.82
Hari ke-6: 13252.03
Hari ke-7: 13244.12
Hari ke-8: 13236.10
Hari ke-9: 13227.95
Hari ke-10: 13219.69
